In [1]:
import pandas as pd
import plotly.express as px
from datetime import datetime
import pytz
import numpy as np

def generate_choropleth_map():
    """
    Generates and displays an interactive choropleth map of global app installs by category.
    The map is displayed only between 6 PM IST and 8 PM IST, filters categories,
    shows top 5 categories, and highlights install thresholds.
    """

    # 1. Time Check: Determine if the current time is within the allowed display window.
    # Set the timezone to Asia/Kolkata (IST)
    ist_timezone = pytz.timezone('Asia/Kolkata')
    current_time_ist = datetime.now(ist_timezone)

    # Define the start and end times for display (6 PM IST to 8 PM IST)
    start_time = current_time_ist.replace(hour=18, minute=0, second=0, microsecond=0)
    end_time = current_time_ist.replace(hour=20, minute=0, second=0, microsecond=0)

    # Check if the current time falls within the specified window
    if not (start_time <= current_time_ist <= end_time):
        print("The dashboard is only available between 6 PM IST and 8 PM IST.")
        return # Exit the function if outside the time window

    # 2. Simulate Data: Create a synthetic dataset for app categories, countries, and installs.
    # In a real-world scenario, you would load your actual data here.
    categories = [
        'Games', 'Education', 'Finance', 'Health & Fitness', 'Shopping',
        'Social', 'Tools', 'Weather', 'Art & Design', 'Communication',
        'Entertainment', 'Food & Drink', 'Travel'
    ]

    # List of countries and their ISO Alpha-3 codes for Plotly
    countries = {
        'USA': 'USA', 'Canada': 'CAN', 'Brazil': 'BRA', 'Germany': 'DEU',
        'France': 'FRA', 'UK': 'GBR', 'India': 'IND', 'China': 'CHN',
        'Australia': 'AUS', 'Japan': 'JPN', 'Mexico': 'MEX', 'South Africa': 'ZAF',
        'Egypt': 'EGY', 'Russia': 'RUS', 'Argentina': 'ARG', 'Indonesia': 'IDN',
        'Nigeria': 'NGA', 'Spain': 'ESP', 'Italy': 'ITA', 'South Korea': 'KOR'
    }

    data = []
    for country_name, iso_alpha3 in countries.items():
        for category in categories:
            # Simulate varying install numbers, with some categories having generally higher installs
            if category in ['Games', 'Social', 'Tools', 'Entertainment']:
                installs = np.random.randint(500000, 5000000) # Higher install range
            else:
                installs = np.random.randint(100000, 2000000) # Lower install range
            data.append([category, country_name, iso_alpha3, installs])

    # Create a Pandas DataFrame from the simulated data
    df = pd.DataFrame(data, columns=['Category', 'Country', 'ISO_Alpha3', 'Installs'])

    # 3. Data Preprocessing and Filtering

    # Filter 3a: Exclude categories that start with 'A', 'C', 'G', or 'S'
    df_filtered_categories = df[~df['Category'].str.startswith(('A', 'C', 'G', 'S'))].copy()

    # Calculate global installs for each category (after applying the initial filter)
    global_installs_by_category = df_filtered_categories.groupby('Category')['Installs'].sum().reset_index()
    global_installs_by_category.rename(columns={'Installs': 'Global_Installs'}, inplace=True)

    # Filter 3b: Determine the top 5 app categories based on global installs
    top_5_categories_df = global_installs_by_category.nlargest(5, 'Global_Installs')
    top_5_categories = top_5_categories_df['Category'].tolist()

    # Filter the main DataFrame to include only these top 5 categories
    df_final = df_filtered_categories[df_filtered_categories['Category'].isin(top_5_categories)].copy()

    # Add a column to indicate if the *global installs for that specific category* exceed 1 million.
    # This will be used in the animation frame label.
    df_final = pd.merge(df_final, global_installs_by_category, on='Category', how='left')
    df_final['Global_Category_Highlight'] = df_final['Global_Installs'].apply(
        lambda x: 'Global Installs > 1M' if x > 1000000 else 'Global Installs <= 1M'
    )

    # Add a column to indicate if the *country-specific installs* for that category exceed 1 million.
    # This will be visible in the hover tooltip.
    df_final['Country_Installs_Highlight'] = df_final['Installs'].apply(
        lambda x: 'Country Installs > 1M' if x > 1000000 else 'Country Installs <= 1M'
    )

    # Combine Category name with its Global_Category_Highlight status for the animation frame label.
    # This makes the global highlighting visible directly on the slider.
    df_final['Category_Display'] = df_final['Category'] + ' (' + df_final['Global_Category_Highlight'] + ')'

    # 4. Plotly Visualization: Create the interactive choropleth map.
    fig = px.choropleth(
        df_final,
        locations="ISO_Alpha3", # Column containing ISO Alpha-3 country codes
        color="Installs",      # Column to determine the color intensity (continuous scale)
        hover_name="Country",  # Column for the country name in the hover tooltip
        hover_data={           # Additional data to show in the hover tooltip
            "Category": True,
            "Installs": True,
            "Global_Category_Highlight": True, # Show global highlight status for the category
            "Country_Installs_Highlight": True, # Show country-specific highlight status
            "ISO_Alpha3": False # Hide ISO_Alpha3 from the hover tooltip
        },
        animation_frame="Category_Display", # Animate the map by each filtered category
        color_continuous_scale=px.colors.sequential.Plasma, # Choose a color scale for install intensity
        scope="world",         # Set the map scope to the entire world
        title="Global App Installs by Top Categories (6PM-8PM IST Only)"
    )

    # Update map layout for better presentation
    fig.update_layout(
        geo=dict(
            showframe=False,      # Do not show map frame
            showcoastlines=False, # Do not show coastlines
            projection_type='equirectangular' # Set projection type
        ),
        margin={"r":0,"t":50,"l":0,"b":0} # Adjust margins
    )

    # Adjust animation speed for better viewing experience
    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1500 # Duration of each frame in ms
    fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 500 # Transition duration in ms

    # Display the figure
    fig.show()

# Call the function to generate and display the map based on conditions
generate_choropleth_map()
